In [2]:
from enum import Enum
from collections import namedtuple
from functools import partial

In [3]:
def BFS(*, start, is_goal, get_neighbors):
    parent = {}
    to_visit = [start]
    discovered = set([start])

    while to_visit:
        vertex = to_visit.pop(0)

        if is_goal(vertex):
            path = []
            while vertex is not None:
                path.insert(0, vertex)
                vertex = parent.get(vertex)
            
            return path
        
        for neighbor in get_neighbors(vertex):
            if neighbor not in discovered:
                discovered.add(neighbor)
                parent[neighbor] = vertex
                to_visit.append(neighbor)

In [4]:
State = namedtuple("State", ["man", "cabbage", "goat", "wolf"])
Location = Enum("Location", ["A", "B"])

In [5]:
start_state = State(
    man = Location.A,
    cabbage = Location.A,
    goat = Location.A,
    wolf = Location.A
)

goal_state = State(
    man = Location.B,
    cabbage = Location.B,
    goat = Location.B,
    wolf = Location.B
)

In [6]:
def is_valid(state):
    goat_eats_cabage = (
        state.goat == state.cabbage
        and state.man != state.goat
    )

    wolf_eats_goat = (
        state.wolf == state.goat
        and state.man != state.wolf
    )

    invalid = goat_eats_cabage or wolf_eats_goat
    
    return not invalid

In [7]:
def next_states(state):
    if state.man == Location.A:
        other_side = Location.B
    else:
        other_side = Location.A

    move = partial(state._replace, man = other_side)

    candidates = [move()]

    for thing in ["cabbage", "goat", "wolf"]:
        if getattr(state, thing) == state.man:
            candidates.append(move(**{thing: other_side}))

    yield from filter(is_valid, candidates)

In [8]:
BFS(
    start = start_state,
    is_goal = goal_state.__eq__,
    get_neighbors = next_states
)

[State(man=<Location.A: 1>, cabbage=<Location.A: 1>, goat=<Location.A: 1>, wolf=<Location.A: 1>),
 State(man=<Location.B: 2>, cabbage=<Location.A: 1>, goat=<Location.B: 2>, wolf=<Location.A: 1>),
 State(man=<Location.A: 1>, cabbage=<Location.A: 1>, goat=<Location.B: 2>, wolf=<Location.A: 1>),
 State(man=<Location.B: 2>, cabbage=<Location.B: 2>, goat=<Location.B: 2>, wolf=<Location.A: 1>),
 State(man=<Location.A: 1>, cabbage=<Location.B: 2>, goat=<Location.A: 1>, wolf=<Location.A: 1>),
 State(man=<Location.B: 2>, cabbage=<Location.B: 2>, goat=<Location.A: 1>, wolf=<Location.B: 2>),
 State(man=<Location.A: 1>, cabbage=<Location.B: 2>, goat=<Location.A: 1>, wolf=<Location.B: 2>),
 State(man=<Location.B: 2>, cabbage=<Location.B: 2>, goat=<Location.B: 2>, wolf=<Location.B: 2>)]

In [ ]:
def describe_solutions(path):
    for old, new in zip(path, path[1:]):
        boat = [thing for thing in ["man", "cabbage", "goat", "wolf"]
                if getattr(old, thing) != getattr(new, thing)    
            ]
        
        print(old.man, "to", new.man, boat)